<a href="https://colab.research.google.com/github/andreasfloros/ARM-ML-Embedded/blob/main/autokeras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import requests
import tarfile

DATASET_ROOT_DIR = 'audio_data/'
os.mkdir(DATASET_ROOT_DIR)
url = 'http://download.tensorflow.org/data/speech_commands_v0.02.tar.gz'
target_path = 'audio_data/dataset.tar.gz'

response = requests.get(url, stream=True)
if response.status_code == 200:
    with open(target_path, 'wb') as f:
        f.write(response.raw.read())

tar = tarfile.open(target_path, "r:gz")
tar.extractall(path='audio_data/')
tar.close()

In [2]:
import os
import librosa
import math
import json
import numpy as np

In [3]:
def make_track_correct_size(signal, expected_num_samples_per_track):

    # print('Original track length: {}'.format(len(signal)))
    # if track is shorter than expected, append it with zeros
    if len(signal) < expected_num_samples_per_track:
      num_zeros_to_pad = expected_num_samples_per_track - len(signal)
      zeros = num_zeros_to_pad * [0.]
      extended_signal = np.append(signal, zeros)
      return extended_signal

    # if track is longer than expected, truncate it
    elif len(signal) > expected_num_samples_per_track:
      return signal[:expected_num_samples_per_track]

    # else return the original track 
    else:
      return signal

In [28]:
def audio_track_to_features(signal, processing_method, sample_rate, window_size, window_stride, num_mfcc):
  
  if processing_method == 'fft':
    # perform Fast Fourier Transform (FFT)
    fft = np.fft.fft(signal)

    # calculate abs values on complex numbers to get magnitude
    spectrum = np.abs(fft)

    # the spectrum is symmetrical with respect to sample_rate / 2
    # so take half of the spectrum and frequency arrays
    # therefore len(half_spectrum) = sample_rate / 2
    half_spectrum = spectrum[:int(len(spectrum)/2)]

    # average every 16 samples to reduce size of array to 1 / 16 of its original size
    # e.g. sample_rate = 16k, duration = 1.024s, reduce size from 8192 to 512 
    averaged = np.mean(half_spectrum.reshape(-1, 16), axis=1)
    return averaged


  elif processing_method == 'stft':
    # perform Short Time Fourier Transform (STFT)
    stft = librosa.stft(signal = signal, 
                        n_fft = window_size, 
                        hop_length = window_stride)

    # calculate abs values on complex numbers to get magnitude
    spectrogram = np.abs(stft)

    # transpose and return the spectrogram matrix
    transposed_spectrogram = spectrogram.transpose()
    return transposed_spectrogram.flatten()


  else: # mfcc
    # perform Mel-Frequency Cepstral Coefficients (MFCC)
    mfcc = librosa.feature.mfcc(signal, 
                                sr = sample_rate, 
                                n_fft = window_size, 
                                n_mfcc = num_mfcc,
                                hop_length = window_stride)
    # transpose and return the mfcc matrix
    transposed_mfcc = mfcc.T
    return transposed_mfcc.flatten()

In [29]:
def preprocess_entire_dataset(dataset_path, json_path, processing_method, sample_rate, expected_duration, window_size, window_stride, num_mfcc):
  # expected duration is in seconds
  expected_num_samples_per_track = int(expected_duration * sample_rate)
  
  # dictionary to later be converted to final json file
  data = {
      'mapping' : [],
      'features' : [],
      'labels' : []
  }

  # we will iterate this for each of the visited sub-directorie in order to
  # give a different label for each of them
  visited_directory_index = 0

  # iterate through all subfolders
  for dirpath, dirnames, filenames in os.walk(dataset_path):

    # # ensure we are not at the dataset root directory
    # # (os.walk provides this directory as well)
    if dirpath is not DATASET_ROOT_DIR:
    # if dirpath == 'audio_data/yes' or dirpath == 'audio_data/no':

      # obtain word labels
      dirpath_components = dirpath.split('/') # audio_data/left => ['audio_data', 'left']
      word_label = dirpath_components[-1]
      data['mapping'].append(word_label)
      print('Processing {}'.format(word_label))

      # access and process files for current word
      for f in filenames:
        
        # load audio file
        file_path = os.path.join(dirpath, f)
        signal, sample_rate = librosa.load(file_path, sr=sample_rate)

        # extend or cut signal to be equal to the expected size
        signal_correct_size = make_track_correct_size(signal, expected_num_samples_per_track)

        # obtain the features of the audio track using the function defined above
        track_features = audio_track_to_features(signal = signal_correct_size, 
                                                 processing_method = processing_method,
                                                 sample_rate = sample_rate, 
                                                 window_size = window_size, 
                                                 window_stride = window_stride, 
                                                 num_mfcc = 13)
        
        # append the audio track features to the features field of the dictionary
        data['features'].append(track_features.tolist())

        # append the current directory index as the label of this track
        data['labels'].append(visited_directory_index)
        # print('file_path: {}'.format(file_path))

      # iterate the index before visiting the next directory
      visited_directory_index = visited_directory_index + 1

  print(data['mapping'])
  print(set(data['labels']))
  # create the json file from the dictionary
  with open(json_path, 'w') as fp:
    json.dump(data, fp, indent=4)

In [30]:
rm -rf `find -type d -name .ipynb_checkpoints`

In [ ]:
!rm audio_data/_background_noise_/README.md

In [ ]:
JSON_PATH = 'data.json'
PROCESSING_METHOD = 'mfcc'
SAMPLE_RATE = 16000
EXPECTED_DURATION = 1.024            # in seconds
WINDOW_SIZE_SAMPLES = 512            # in samples
WINDOW_STRIDE_SAMPLES = 320          # in samples
MFCC_COEFF_NUMBER = 13

preprocess_entire_dataset(dataset_path = DATASET_ROOT_DIR, 
                   json_path = JSON_PATH, 
                   processing_method = PROCESSING_METHOD,
                   sample_rate = SAMPLE_RATE, 
                   expected_duration = EXPECTED_DURATION, 
                   window_size = WINDOW_SIZE_SAMPLES, 
                   window_stride = WINDOW_STRIDE_SAMPLES, 
                   num_mfcc = MFCC_COEFF_NUMBER)

Import libraries for training

In [ ]:
!pip install autokeras

In [57]:
import json
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Reshape, Activation, BatchNormalization, Conv2D, MaxPooling2D, Flatten
import matplotlib.pyplot as plt
import autokeras as ak
import kerastuner as kt

Load the data and prepare datasets

In [12]:
def load_data(data_path):
    """Loads training dataset from json file.
        :param data_path (str): Path to json file containing data
        :return X (ndarray): Inputs
        :return y (ndarray): Targets
    """

    print('Loading dataset')

    with open(data_path, "r") as fp:
        data = json.load(fp)

    X = np.array(data['features'])
    y = np.array(data['labels'])
    return X, y


def prepare_datasets(test_size, validation_size):
    """Loads data and splits it into train, validation and test sets.
    :param test_size (float): Value in [0, 1] indicating percentage of data set to allocate to test split
    :param validation_size (float): Value in [0, 1] indicating percentage of train set to allocate to validation split
    :return X_train (ndarray): Input training set
    :return X_validation (ndarray): Input validation set
    :return X_test (ndarray): Input test set
    :return y_train (ndarray): Target training set
    :return y_validation (ndarray): Target validation set
    :return y_test (ndarray): Target test set
    """

    print('Splitting dataset into training, validation, and test splits')

    # load data
    X, y = load_data(JSON_PATH)

    # create train, validation and test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    # X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=validation_size)

    # add an axis to input sets for conv networks to fit conv2D shape specs
    # currently not needed since we reshape everything anyway
    # X_train = X_train[..., np.newaxis]
    # X_validation = X_validation[..., np.newaxis]
    # X_test = X_test[..., np.newaxis]

    return X_train, X_test, y_train, y_test

In [ ]:
X_train, X_test, y_train, y_test = prepare_datasets(0.25, 0.2)
print('Finished preparing training, validation, and test data')
print('X_train.shape: {}'.format(X_train.shape))

Build AutoKera Model

In [73]:
hp = kt.HyperParameters()
num_units = hp.Choice("num_units", values=[16,32,64])
input_node = ak.Input()
output_node = ak.DenseBlock()(input_node)
output_node = ak.ClassificationHead(num_classes=36, loss = "categorical_crossentropy")(output_node)
auto_model = ak.AutoModel(
    inputs=input_node, outputs=output_node, objective="accuracy", tuner="greedy", overwrite=True, max_trials=10
)
auto_model.fit(X_train, y_train, epochs=10)

Trial 10 Complete [00h 01m 28s]
accuracy: 0.7838003635406494

Best accuracy So Far: 0.7838003635406494
Total elapsed time: 00h 11m 28s
INFO:tensorflow:Oracle triggered exit
Epoch 1/10
2481/2481 [==============================] - 10s 3ms/step - loss: 3.1792 - accuracy: 0.1637
Epoch 2/10
2481/2481 [==============================] - 9s 4ms/step - loss: 1.5075 - accuracy: 0.5591
Epoch 3/10
2481/2481 [==============================] - 9s 3ms/step - loss: 1.2067 - accuracy: 0.6423
Epoch 4/10
2481/2481 [==============================] - 9s 4ms/step - loss: 1.0683 - accuracy: 0.6821
Epoch 5/10
2481/2481 [==============================] - 9s 3ms/step - loss: 0.9757 - accuracy: 0.7081
Epoch 6/10
2481/2481 [==============================] - 9s 4ms/step - loss: 0.9055 - accuracy: 0.7289
Epoch 7/10
2481/2481 [==============================] - 9s 4ms/step - loss: 0.8470 - accuracy: 0.7462
Epoch 8/10
2481/2481 [==============================] - 8s 3ms/step - loss: 0.7958 - accuracy: 0.7614
Epoch 9/10

In [74]:
print(auto_model.evaluate(X_test, y_test))

827/827 [==============================] - 2s 2ms/step - loss: 0.8010 - accuracy: 0.7621
[0.800967276096344, 0.7621225118637085]


In [76]:
model = auto_model.export_model()
print(type(model))

<class 'tensorflow.python.keras.engine.functional.Functional'>


In [77]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 676)]             0         
_________________________________________________________________
cast_to_float32 (CastToFloat (None, 676)               0         
_________________________________________________________________
dense (Dense)                (None, 32)                21664     
_________________________________________________________________
batch_normalization (BatchNo (None, 32)                128       
_________________________________________________________________
re_lu (ReLU)                 (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               16896     
_________________________________________________________________
batch_normalization_1 (Batch (None, 512)               2048  